In [3]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import joblib
import yfinance as yf

import src
cloudpickle.register_pickle_by_value(src) # for model portability

In [2]:

# ticker = yf.Ticker("1301.T")
# # print(ticker.info)
# display(ticker.history(period="max"))

# display(ticker.actions)
# display(ticker.dividends)
# display(ticker.splits)
# display(ticker.financials)
# display(ticker.quarterly_financials)
# display(ticker.major_holders)
# display(ticker.institutional_holders)
# display(ticker.balance_sheet)
# display(ticker.quarterly_balance_sheet)

# display(ticker.cashflow)
# display(ticker.quarterly_cashflow)
# display(ticker.earnings)
# display(ticker.quarterly_earnings)

# display(ticker.sustainability)
# display(ticker.recommendations)
# display(ticker.calendar)
# display(ticker.earnings_dates)
# display(ticker.isin)
# display(ticker.options)
# display(ticker.news)

# # get option chain for specific expiration
# # opt = msft.option_chain('YYYY-MM-DD')
# # data available via: opt.calls, opt.puts

In [9]:
df_code = read_code()
# display(df_code)

df = yf.download(['{}.T'.format(x) for x in df_code['symbol'].iloc[:100]])
df = df.stack()
df = df.reset_index()
display(df)

[*********************100%***********************]  100 of 100 completed


,Date,level_1,Adj Close,Close,High,Low,Open,Volume
0,2000-01-04,1332.T,102.602020,157.0,165.0,157.0,158.0,296000.0
1,2000-01-04,1380.T,473.627441,605.0,625.0,605.0,605.0,0.0
2,2000-01-04,1418.T,1031.356079,1250.0,1250.0,1250.0,1250.0,5000.0
3,2000-01-05,1332.T,102.602020,157.0,170.0,156.0,164.0,637000.0
4,2000-01-05,1380.T,473.627441,605.0,615.0,605.0,605.0,0.0
...,...,...,...,...,...,...,...,...
273714,2023-03-14,1473.T,1967.000000,1967.0,1988.5,1958.0,1988.5,107890.0
273715,2023-03-14,1474.T,17660.000000,17660.0,17845.0,17600.0,17845.0,2029.0
273716,2023-03-14,1475.T,1987.000000,1987.0,2004.0,1973.0,2004.0,690042.0
273717,2023-03-14,1476.T,1884.000000,1884.0,1894.0,1856.0,1871.0,92916.0


In [4]:
df = yf.download(['1301.T', '1312.T'])
display(df)

[*********************100%***********************]  2 of 2 completed


Adj Close            Close             High              Low  \
                 1301.T   1312.T  1301.T   1312.T  1301.T   1312.T  1301.T   
Date                                                                         
2001-01-01   956.234619      NaN  1400.0      NaN  1400.0      NaN  1400.0   
2001-01-02   956.234619      NaN  1400.0      NaN  1400.0      NaN  1400.0   
2001-01-03   956.234619      NaN  1400.0      NaN  1400.0      NaN  1400.0   
2001-01-04   949.404114      NaN  1390.0      NaN  1420.0      NaN  1370.0   
2001-01-05   908.422913      NaN  1330.0      NaN  1400.0      NaN  1330.0   
...                 ...      ...     ...      ...     ...      ...     ...   
2023-03-08  3660.000000  21615.0  3660.0  21615.0  3665.0  21615.0  3640.0   
2023-03-09  3665.000000  21615.0  3665.0  21615.0  3675.0  21615.0  3655.0   
2023-03-10  3625.000000      NaN  3625.0      NaN  3665.0      NaN  3615.0   
2023-03-13  3550.000000      NaN  3550.0      NaN  3610.0      NaN  3530.0   
2023-03-14  3460.000000      NaN  3460.0      NaN  3530.0      NaN  3455.0   

                       Open          Volume         
             1312.T  1301.T   1312.T 1301.T 1312.T  
Date                                                
2001-01-01      NaN  1400.0      NaN      0    NaN  
2001-01-02      NaN  1400.0      NaN      0    NaN  
2001-01-03      NaN  1400.0      NaN      0    NaN  
2001-01-04      NaN  1420.0      NaN  19300    NaN  
2001-01-05      NaN  1390.0      NaN  19700    NaN  
...             ...     ...      ...    ...    ...  
2023-03-08  21615.0  3640.0  21615.0  22000    0.0  
2023-03-09  21615.0  3660.0  21615.0  33200    0.0  
2023-03-10      NaN  3650.0      NaN  74000    NaN  
2023-03-13      NaN  3610.0      NaN  90800    NaN  
2023-03-14      NaN  3530.0      NaN  81000    NaN  

[5545 rows x 12 columns]

In [4]:
df = yf.download(['1001.T'])
display(df)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 1001.T: No timezone found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [10]:
import time
from datetime import datetime


def read_code():
    url = 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls'
    df = pd.read_excel(url)
    df = df.rename(columns={
        'コード': 'symbol',
        '銘柄名': 'symbol_name',
        '33業種コード': 'industry_code',
    })
    df['symbol_name'] = df['symbol_name'].str.normalize('NFKC')
    df['industry_code'] = df['industry_code'].astype(str).str.replace('-', '0').astype(int)
    return df

class Fetcher:
    def __init__(self):
        self.keys = {}
        self.data_id = 'stock_ohlcv'
        
    def fetch(self, last_timestamp=None):
        if last_timestamp is not None and time.time() < last_timestamp + (24 + 9) * 60 * 60:
            print('no data')
            return pd.DataFrame()
        
        df_code = read_code()
        
        if last_timestamp is None:
            start = '1900-01-01'
        else:
            start = datetime.fromtimestamp(last_timestamp + 24 * 60 * 60).strftime('%Y-%m-%d')
        
        df = yf.download(['{}.T'.format(x) for x in df_code['symbol']], start=start)
        df = df.stack()
            
        if df.shape[0] == 0:
            return pd.DataFrame()
            
        df = df.reset_index()
        df = df.rename(columns={
            'Date': 'timestamp',
            'level_1': 'symbol',
            'Open': 'op',
            'High': 'hi',
            'Low': 'lo',
            'Close': 'cl',
            'Adj Close': 'adj_cl',
            'Volume': 'volume',
        })
        df['timestamp'] = df['timestamp'].astype(int) // 10 ** 9
        df = df.set_index('timestamp')
        
        df['symbol_name'] = df_code['symbol_name'].iloc[0]
        df['industry_code'] = df_code['industry_code'].iloc[0]
        
        if last_timestamp is not None:
            df = df.loc[df.index > last_timestamp]
        
        return df


In [5]:
fetcher = Fetcher()
df = fetcher.fetch(last_timestamp=None)
display(df)
df = fetcher.fetch(last_timestamp=1600041600)
display(df)

NameError: name 'Fetcher' is not defined

In [11]:
fetchers = [Fetcher()]

data = cloudpickle.dumps(fetchers)
data = lzma.compress(data)
with open('/home/jovyan/data/20230313_stock.xz', 'wb') as f:
    f.write(data)